In [1]:
!git clone https://github.com/haoweiwang0/Coreset_Prioritization.git
%cd Coreset_Prioritization

Cloning into 'Coreset_Prioritization'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 37 (delta 16), reused 21 (delta 8), pack-reused 0
Receiving objects: 100% (37/37), 579.56 KiB | 2.17 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/Coreset_Prioritization


In [5]:
!pip install neural-tangents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.7/248.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 6.8 MB/s eta 0:00:00


In [6]:
!pip install -U "jax[cuda12]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.6/81.6 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.3/363.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.7/895.7 kB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.6/574.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.6/217.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
import bilevel_coreset
import loss_utils

import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import torch.nn.functional as F

subset_size = 50

def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(0)

transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

train_dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=transform_test)
test_dataset = datasets.CIFAR10(root='data', train=False, download=False, transform=transform_test)

100%|██████████| 170498071/170498071 [00:04<00:00, 41652311.15it/s]


Extracting data/cifar-10-python.tar.gz to data


In [3]:
from tqdm.auto import tqdm
import models

def train_model(model, loader):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=0)
    pbar = tqdm(range(nr_epochs), desc="Training", unit="epoch")

    for epoch in pbar:
        model.train()
        training_loss = 0.0

        for batch_idx, (data, target) in enumerate(loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.cross_entropy(output, target)
            loss.backward()
            training_loss += loss.item()
            optimizer.step()

        pbar.set_postfix({'loss': training_loss / (batch_idx + 1)})

def test_model(model, loader):
    model.to(device)
    criterion = torch.nn.CrossEntropyLoss()

    model.eval()
    pbar = tqdm(loader, desc="Testing", unit="batch")
    correct = 0
    testing_loss = 0.0

    with torch.no_grad():
        for data, target in pbar:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            testing_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            pbar.set_postfix({'loss': testing_loss / len(loader)})
    test_acc = 1. * correct / len(loader.dataset)
    return test_acc

In [4]:
nr_classes = 10
batch_size = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'
nr_epochs = 50

lenet5 = models.ConvNet_cifar(output_dim=nr_classes).to(device)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, drop_last=True)

train_model(lenet5, train_loader)
print('Original LeNet - Train accuracy', test_model(lenet5, train_loader))
print('Original LeNet - Test accuracy', test_model(lenet5, test_loader))

Training:   0%|          | 0/50 [00:00<?, ?epoch/s]

Testing:   0%|          | 0/1000 [00:00<?, ?batch/s]

Original LeNet - Train accuracy 0.63506


Testing:   0%|          | 0/200 [00:00<?, ?batch/s]

Original LeNet - Test accuracy 0.609


In [9]:
from jax import jit
from neural_tangents import stax
import neural_tangents as nt

_, _, kernel_fn = stax.serial(
        stax.Conv(6, (5, 5), (1, 1), padding='SAME', W_std=1., b_std=0.05),
        stax.Relu(),
        stax.Conv(16, (5, 5), (1, 1), padding='SAME', W_std=1., b_std=0.05),
        stax.Relu(),
        stax.Flatten(),
        stax.Dense(120, 1., 0.05),
        stax.Relu(),
        stax.Dense(84, 1., 0.05),
        stax.Relu(),
        stax.Dense(10, 1., 0.05))
kernel_fn = jit(kernel_fn, static_argnums=(2,))

def generate_cnn_ntk(X, Y):
    n = X.shape[0]
    m = Y.shape[0]
    K = np.zeros((n, m))
    for i in range(m):
        K[:, i:i + 1] = np.array(kernel_fn(X, Y[i:i + 1], 'ntk'))
    return K

proxy_kernel_fn = lambda x, y: generate_cnn_ntk(x.transpose(0, 2, 3, 1), y.transpose(0, 2, 3, 1))

In [10]:
# work only on the first 10000 samples for speedup

limit = 10000

loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)
X, y = next(iter(loader))
X, y = X.numpy(), y.numpy()
all_inds = np.random.choice(np.arange(len(train_dataset)), 10000, replace=False)
X = X[all_inds]
y = y[all_inds]

bc = bilevel_coreset.BilevelCoreset(outer_loss_fn=loss_utils.cross_entropy,
                                    inner_loss_fn=loss_utils.cross_entropy, out_dim=10,
                                    max_outer_it=10, outer_lr=0.05, max_inner_it=200)
coreset_inds, _ = bc.build_with_representer_proxy_batch(X, y, subset_size, proxy_kernel_fn, cache_kernel=True,
                                                        start_size=10, inner_reg=1e-7)

Coreset size 10, outer_loss 4.79, inner loss 0.000425
Coreset size 20, outer_loss 2.6, inner loss 0.000765
Coreset size 30, outer_loss 2.26, inner loss 0.00109
Coreset size 40, outer_loss 2.18, inner loss 0.00137
Coreset size 50, outer_loss 2.11, inner loss 0.00165


In [11]:
nr_epochs = 1000

coreset_net = models.ConvNet_cifar(output_dim=nr_classes).to(device)
coreset_subset = Subset(train_dataset, coreset_inds)
train_loader = torch.utils.data.DataLoader(coreset_subset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

train_model(coreset_net, train_loader)
print('Coreset LeNet - Train accuracy', test_model(coreset_net, train_loader))
print('Coreset LeNet - Test accuracy', test_model(coreset_net, test_loader))

Training:   0%|          | 0/1000 [00:00<?, ?epoch/s]

Testing:   0%|          | 0/1 [00:00<?, ?batch/s]

Coreset LeNet - Train accuracy 1.0


Testing:   0%|          | 0/200 [00:00<?, ?batch/s]

Coreset LeNet - Test accuracy 0.1815


In [13]:
def test_with_logits(model, coreset_model, loader):
    model.eval()
    logits = []
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = F.softmax(model(data), dim=1)
            coreset_output = F.softmax(coreset_model(data), dim=1)
            logits.append((data, output, coreset_output, target))

    return logits

logits = test_with_logits(lenet5, coreset_net, test_loader)

In [14]:
prioritized_test_data = []

# calculate the similarity score between the logits from
# original DNN and coreset-trained DNN
# reorder test cases based on the similarity score
def test_case_prioritization(logits):
    result = []
    probs = torch.cat([item[1] for item in logits], dim=0).cpu().numpy()
    coreset_probs = torch.cat([item[2] for item in logits], dim=0).cpu().numpy()
    inputs = torch.cat([item[0] for item in logits], dim=0).cpu().numpy()
    targets = torch.cat([item[3] for item in logits], dim=0).cpu().numpy()
    for i in range(probs.shape[0]):
        item = probs[i]
        coreset_item = coreset_probs[i]
        similarity_score = F.cosine_similarity(torch.tensor(item).unsqueeze(0), torch.tensor(coreset_item).unsqueeze(0))
        result.append({'similarity_score': similarity_score,
                       'probabilities': item,
                       'input': inputs[i],
                       'target': targets[i]})
    result = sorted(result, key=lambda x: x['similarity_score'], reverse=False)
    return result

In [15]:
prioritized_test_data = test_case_prioritization(logits)

In [16]:
from torch.utils.data import TensorDataset

inputs = torch.stack([torch.from_numpy(item['input']) for item in prioritized_test_data])
targets = torch.stack([torch.tensor(item['target']) for item in prioritized_test_data])
tensor_dataset = TensorDataset(inputs, targets)
prioritized_loader = torch.utils.data.DataLoader(tensor_dataset, batch_size=1, shuffle=False)

In [18]:
def test_with_APFD(model, loader):
    model.to(device)
    model.eval()

    TFs = 10 * [0]

    with torch.no_grad():
        for ids, (data, target) in enumerate(loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            category = output.argmax(dim=1, keepdim=True)
            if category != target and TFs[target] == 0:
                TFs[target] = ids + 1

            if all(TFs):
                break

    APFD = 1 - (sum(TFs) / (10 * len(loader))) + 1 / (2 * len(loader))

    return APFD

print('LeNet - APFD', test_with_APFD(lenet5, prioritized_loader))

LeNet - APFD 0.9906
